# Credmark Modeling Framework Example for Jupyter notebook
## Introduction

version: 2022.6.3

In [1]:
from credmark.cmf.engine.model_loader import ModelLoader
from credmark.cmf.engine.context import EngineModelContext
from credmark.cmf.model.context import ModelContext

from credmark.dto import *
from credmark.cmf.types import *

from credmark.cmf.types.ledger import (
    BlockTable, ContractTable,
    LogTable, ReceiptTable, TokenTable, TokenTransferTable,
    TraceTable, TransactionTable, LedgerTable,
    LedgerAggregate, LedgerModelOutput
)

from credmark.cmf.engine.dev_models.console import get_dt, get_block, log_output

## Initialize

1. You could change the level of logging with `log_output()`.

In [2]:
import logging
# Change level of logging from the default of WARNING, output unchanged to stream
log_output(log_level=logging.INFO)

# Change output of logging to a file, and change level to DEBUG
# logging_output('../tmp/debug.log')

2022-06-07 12:01:10,280 - credmark.cmf.engine.context - INFO - Enabled log with level=INFO


2. Create context and some shortcuts for frequently used utilitis.

Update below parameters
- chain_id: 1 for ETH
- block_number: None or a specific number.
- chain_to_provider_url: web3 node (archive is preferred)
- api_url: None or a specific gateway server address
- console: True
- use_local_models: None, '*', or a comma-separated list.

In [3]:
model_loader = ModelLoader(['../models'], None, True)

params = {'chain_id': 1,
          'block_number': None,
          'model_loader': model_loader,
          'chain_to_provider_url': {'1': 'http://localhost:10444'},
          'api_url': None,
          'run_id': None,
          'console': True,
          'use_local_models': None # '*' # 'token.price'
         }
context = EngineModelContext.create_context(**params)

ledger = context.ledger
run_model = context.run_model
models = context.models
block_number = context.block_number
chain_id = context.chain_id
web3 = context.web3
run_model_historical = context.historical.run_model_historical
run_model_historical_blocks = context.historical.run_model_historical_blocks

2022-06-07 12:01:12,105 - credmark.cmf.engine.context - INFO - Using latest block number 14921846


## Use Cmf

### 1. Basic utilities

In [4]:
get_dt(2022, 5, 3)

datetime.datetime(2022, 5, 3, 0, 0, tzinfo=datetime.timezone.utc)

In [5]:
get_block(get_dt(2022, 5, 3))

14701368

In [6]:
context.block_number

14921846

In [7]:
context.chain_id

1

In [8]:
context.web3

In [9]:
context.web3.eth.get_balance('0xd3CdA913deB6f67967B99D67aCDFa1712C293601')

1790191567590102228

In [10]:
context.web3.eth.get_block(context.block_number)['gasUsed']

8121251

### 2. Use models

1. Type `models.` and hit `TAB` key, a list of models will be pop up.
2. Select a model and type ? will show description and help, `models.token.info?`
3. Show the list of models wiht `dir(models)`, or `model_loader.loaded_model_versions()` with version information.
4. If you have updated any model in the folder on disk, run `model_loader.reload()` to refresh.

In [11]:
dir(models)

['aave_v2.get_lending_pool',
 'aave_v2.get_lending_pool_provider',
 'aave_v2.get_lending_pool_providers_from_registry',
 'aave_v2.get_oracle_price',
 'aave_v2.get_price_oracle',
 'aave_v2.lending_pool_assets',
 'aave_v2.overall_liabilities_portfolio',
 'aave_v2.token_asset',
 'aave_v2.token_liability',
 'account.portfolio',
 'account.portfolio_aggregate',
 'chainlink.get_feed_registry',
 'chainlink.price_by_ens',
 'chainlink.price_by_feed',
 'chainlink.price_by_registry',
 'chainlink.price_usd',
 'cmk.circulating_supply',
 'cmk.get_all_vesting_balances',
 'cmk.get_vesting_accounts',
 'cmk.get_vesting_info_by_account',
 'cmk.total_supply',
 'cmk.vesting_contracts',
 'cmk.vesting_events',
 'compound_v2.all_pools_info',
 'compound_v2.all_pools_values',
 'compound_v2.get_comptroller',
 'compound_v2.get_pool_info',
 'compound_v2.get_pools',
 'compound_v2.pool_value',
 'compound_v2.pool_value_historical',
 'console',
 'contract.event_data',
 'contract.function_data',
 'contract.metadata',
 '

In [12]:
models.token.info?

Call signature:
models.token.info(
    input: Union[pydantic.main.BaseModel, dict, NoneType] = None,
    return_type: Union[dict, Type[pydantic.main.BaseModel], NoneType] = None,
    **kwargs,
) -> Union[dict, pydantic.main.BaseModel]
Type:           RunModelMethod
String form:    <credmark.cmf.model.context.RunModelMethod object at 0x7fd33944beb0>
File:           ~/mambaforge/envs/cm310-2/lib/python3.10/site-packages/credmark/cmf/model/context.py
Docstring:     
token.info
- slug: token.info
- displayName: Token Information
- description: Return token's information
- latestVersion: 1.1
- developer: Credmark
- input schema (* for required field):
  Token(Token(*))
    └─address(string)
- input example:
  #01: {"address": "0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9"}
  #02: {"symbol": "AAVE"}
  #03: {"address": "0x1F98431c8aD98523631AE4a59f267346ea31F984"}
  #04: {"address": "0x1F98431c8aD98523631AE4a59f267346ea31F984", "abi": "(Optional) contract abi JSON string"}
- output schema (* fo

In [13]:
model_loader.reload()

## 3. Create and use types

### 3.1 Example - get price for USDC

In [14]:
usdt = Token(address='0xdAC17F958D2ee523a2206206994597C13D831ec7')
print('USDT decimals:', usdt.decimals)
models.chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

USDT decimals: 6


{'price': 0.99950262,
 'src': 'chainlink.price-by-feed|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|None|t:16462s|r:0'}

In [15]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

{'src': 'chainlink.price-by-feed|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|None|t:17407s|r:0',
 'price': 1.00010233}

In [16]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_usd(usdt)

{'src': 'chainlink.price-by-registry|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|True|t:17407s|r:0',
 'price': 1.00010233}

### 3.1 Run a model

#### Run as of current block

In [17]:
aave = Token(symbol='AAVE')

print(block_number,
      block_number.timestamp_datetime,
      'models:', context.models().chainlink.price_usd(input=aave, return_type=Price).price,
      'run_model:', context.run_model('chainlink.price-usd', input=aave, return_type=Price).price)

14921846 2022-06-07 16:27:03+00:00 models: 98.85740049 run_model: 98.85740049


#### Run as of past blocks

In [18]:
context.block_number.timestamp_datetime

datetime.datetime(2022, 6, 7, 16, 27, 3, tzinfo=datetime.timezone.utc)

In [19]:
for dt in [get_dt(2022, 5, 3), get_dt(2022, 5, 9), get_dt(2022, 1, 1)]:
    block = get_block(dt)
    print(block,
          block.timestamp_datetime,
          'models:', context.models(block).chainlink.price_usd(input=aave, return_type=Price).price,
          'run_model:', context.run_model('chainlink.price-usd', input=aave, return_type=Price, block_number=block).price)

14701368 2022-05-02 23:59:50+00:00 models: 144.94887195 run_model: 144.94887195
14739154 2022-05-08 23:59:54+00:00 models: 125.63830024 run_model: 125.63830024
13916165 2021-12-31 23:59:49+00:00 models: 255.01691343 run_model: 255.01691343


### 3.2 Travel between different blocks

Tavel is one-way only - can only travel to earlier/smaller block numbers, and not to the future/later blocks.

In [20]:
context.block_number = get_block(get_dt(2022, 5, 3))
models.chainlink.price_usd(input=aave, return_type=Price).price

144.94887195

In [21]:
# Below will gave any error as we wants to travel back
# context.block_number = get_block(get_dt(2022, 5, 30))
# models.chainlink.price_usd(input=aave, return_type=Price).price

Instead, create a new context. It will automatically become the current context.

In [22]:
context_old = context.create_context(**params | {'block_number': get_block(get_dt(2022, 4, 3))})
context_old.block_number.timestamp_datetime, context.block_number.timestamp_datetime

(datetime.datetime(2022, 4, 2, 23, 59, 59, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 5, 2, 23, 59, 50, tzinfo=datetime.timezone.utc))

If we would like to use previously defined context, run `context.set_current()` to set it back to the current context.

Otherwise, we may encounter error with running model for a block number later than the context.

In [23]:
context.set_current()

### 4. Ledger

In [24]:
df_ledger = (context.ledger.get_transactions(columns=[TransactionTable.Columns.BLOCK_HASH,
                                          TransactionTable.Columns.FROM_ADDRESS,
                                          TransactionTable.Columns.TO_ADDRESS,
                                          TransactionTable.Columns.VALUE],
                                 where=f'{TransactionTable.Columns.BLOCK_NUMBER} = {context.block_number-1000}',
                                 order_by=TransactionTable.Columns.BLOCK_TIMESTAMP,
                                 limit='5')
             .to_dataframe())
df_ledger

,block_hash,from_address,to_address,value
0,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xf16e9b0d03470827a95cdfd0cb8a8a3b46969b91,0x5218e472cfcfe0b64a064f055b43b4cdc9efd3a6,0
1,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0x8c6f71f96419325c2f637332650029e2a53787a4,0xd19053361e356e79166b22beaacd248b1b63135f,5294873127907440
2,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0x8fa7b1d8dffe8c2a94ad15806110c24591e1153f,0xf598b81ef8c7b52a7f2a89253436e72ec6dc871f,97898558144736000
3,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xec30d02f10353f8efc9601371f56e808751f396f,0xdac17f958d2ee523a2206206994597c13d831ec7,0
4,0x8ccec7255f1c2519f87936a725a957d246f21047e676...,0xcad621da75a66c7a8f4ff86d30a2bf981bfc8fdd,0xebd478795b54ae755b3eb1932885453ab43fee84,3679563370000000000
